In [1]:
import pandas as pd
from logging import getLogger


In [2]:
logger = getLogger(__name__)

In [4]:
product_df = pd.read_pickle('../data/processed/openfoodfacts.pkl')

In [5]:
product_df.shape

(293311, 44)